# Use the prediction endpoint
This notebook demonstrates how use the REST prediction endpoint to remotely access a custom image classification model you trained in the previous step of the lab. 

Note that the prediction endpoint is a REST web service and as such can be easily accessed using any tool and programming environment supporting REST interfaces. For example you can invoke the service using `curl` utility in Linux.


## Download test images

In [ ]:
%%sh
cd
mkdir samples
cd samples
wget https://github.com/jakazmie/AIDays/raw/master/DeveloperTrack/01-CustomVisionService/samples/barren-1.png
wget https://github.com/jakazmie/AIDays/raw/master/DeveloperTrack/01-CustomVisionService/samples/cultivated-1.png
wget  https://github.com/jakazmie/AIDays/raw/master/DeveloperTrack/01-CustomVisionService/samples/developed-1.png
ls -l   
    

## Get the prediction key and project id
Before you can access the endpoint you need to retrieve project id and prediction key. If you want to access a specific iteration you will also need Iteration ID. If you don't specify Iteration ID the call will be routed to the default iteration.

To get the keys, navigate to the Custom Vision web page and select the gear icon in the upper right. In the Accounts section, copy the value from the Prediction Key field.

![Keys](https://github.com/jakazmie/AIDays/raw/master/DeveloperTrack/01-CustomVisionService/images/img13.PNG)


Change the values below to your project's keys


In [ ]:
%env PROJECT_ID=<your project ID>
%env PREDICTION_KEY=<your prediction key>

PROJECT_ID = %env PROJECT_ID
PREDICTION_KEY = %env PREDICTION_KEY

If you want to try to invoke the prediction point using `curl`, modify the below `curl` command by inserting your `Project ID` and `Prediction Key`.

In [ ]:
%%sh

curl -X POST https://southcentralus.api.cognitive.microsoft.com/customvision/v2.0/Prediction/$PROJECT_ID/image -H "Prediction-Key: $PREDICTION_KEY"  -H "Content-Type: application/octet-stream" --data-binary @samples/developed-1.png

To simplify integration with your applications, Custom Vision Service provides SDKs that wrap REST APIs, including the prediction endpoint, in easy to use libraries. Currently, C#, Go, and Python SDKs are available.


## Install Custom Vision Service SDK

Before proceeding we will install Custom Vistion Service SDK from PyPI. If you are running this notebook in the Python environment that has the SDK pre-installed you can skip this step.

In [ ]:
# Install Custom Vision Service SDK  in the current Jupyter kernel
import sys
!{sys.executable} -m pip install azure-cognitiveservices-vision-customvision


## Display test images

The images we will use for testing are located in the `samples` folder.

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline

images_dir = 'samples'
images = [os.path.join(images_dir, file) for file in os.listdir(images_dir)]

figsize=(10, 8)
fig, axis = plt.subplots(len(images)//3, 3, figsize=figsize)
fig.tight_layout()
for ax, image_path in zip(axis.flat[0:], images):
    image = Image.open(image_path)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.imshow(image)

## Call the prediction endpoint

Python SDK wraps the prediction endpoint in the `prediction_endpoint` class. The class exposes the `predict_image` method that takes a Python File Object as parameter. The following code snippet defines a utility function `classify_image` that invokes the prediction endpoint and parses the results returned from the service.

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

def classify_image(image_path):
    predictor = prediction_endpoint.PredictionEndpoint(PREDICTION_KEY)
    with open(image_path, mode='rb') as image:
      result = predictor.predict_image(PROJECT_ID, image)
    
    probs = [prediction.probability for prediction in result.predictions]
    max_prob = max(probs)
    max_index = probs.index(max_prob)
    tag = result.predictions[max_index].tag_name

    return tag, max_prob
    

We will now invoke the prediction endpoint and display the results returned by the service.

In [ ]:
figsize=(10, 8)
fig, axis = plt.subplots(len(images)//3, 3, figsize=figsize)
fig.tight_layout()
for ax, image_path in zip(axis.flat[0:], images):
    tag, prob = classify_image(image_path)
    ax.set_title(tag + ': ' + str(prob))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    image = Image.open(image_path)
    ax.imshow(image)

